In [6]:
import numpy as np
import pandas as pd
import scipy.linalg as la
from numpy.linalg import matrix_power
import matplotlib.pyplot as plt
import heapq
import sys
import math

# the object for Customer
# You have to change the object Customer
class Customer:
    # key is arrival time / departure time
    def __init__(self, ID, arrival_time, dining_time):
        self.ID = ID
        self.arrival_time = arrival_time
        self.dining_time = dining_time
        self.entry_time = None
        self.departure_time = -1
        self.waiting_time = None
        self.sID = None
        self.angryDay = 0
        self.isInside = False
        self.isFinished = False
        self.isWaiting = False
        self.isAngry = 0
        self.isRemoved = 0
        self.isBlacklisted = 0
        
    def __lt__(self,other):
        return self.ID < other.ID
    
    def reset(self, ID, arrival_time):
        self.arrival_time = arrival_time
        self.entry_time = None
        self.departure_time = -1
        self.waiting_time = None
        self.sID = None
        self.isInside = False
        self.isFinished = False
        self.isWaiting = False

# the object for Server
# 1 Server = 1 Table
# You have to change the object Server
class Server:
    def __init__(self, ID, size):
        self.idle = True
        self.current_time = 0
        self.ID = ID
        self.serving = -1
        self.size = size
        self.cIDarray = []
    
    
    def set_busy(self, cID, departure_time):
        self.idle = False
        self.serving = cID
        self.current_time = departure_time
        
    def set_idle(self):
        self.idle = True
        self.serving = -1

def timing_routine(customer_list, customer_queue, server_list, curr_time, termination_time, curr_day):
    global total_server
    global nFinish
    global record_result
    # determine time of next event
    # please take referece to the code in introduction to basic queueing system
    # step 0: you need to change the argument list above
    # step 1: determine the time based on customer_list
    # step 2: determine the time based on the queueing
    # step 3: determine the time based on the servers (tables)
    
    # for 100 customer
    for i in np.arange(N):
        # leave table
        if(customer_list[i].departure_time == curr_time):
            print("Customer " + str(customer_list[i].ID) + " is leaving at " + str(customer_list[i].departure_time))            
            server_list[customer_list[i].sID].cIDarray.remove(customer_list[i].ID)
            server_list[customer_list[i].sID].size=server_list[customer_list[i].sID].size + 1
            total_server = total_server + 1
            print("Remaining table is " + str(total_server))
            nFinish = nFinish + 1
            event_dict = pd.DataFrame({"Day":[curr_day],"Time":[curr_time], "Event":["Customer " + str(customer_list[i].ID) + " dined at table " + str(customer_list[i].sID) + " and departs"], "len(Queue)":[len(customer_queue)], "Number_dining":[36-total_server], "Number_finish":[nFinish]})
            record_result = record_result.append(event_dict, ignore_index=True)
        
        # closed hour
        if(curr_time >= termination_time):
            continue
        
        # wait outside in a queue
        if(customer_list[i].arrival_time == curr_time):
            customer_list[i].isWaiting = True
            customer_queue.append(customer_list[i])
            # arrival record
            event_dict = pd.DataFrame({"Day":[curr_day],"Time":[curr_time], "Event":["Customer " + str(customer_list[i].ID) + " arrives"], "len(Queue)":[len(customer_queue)-1], "Number_dining":[36-total_server], "Number_finish":[nFinish]})
            record_result = record_result.append(event_dict, ignore_index=True)
            
        # enter restaurant
        if(len(customer_queue)>0 and total_server > 0 and total_server <= 36):
            # check if customer is first in the queue
            if(customer_list[i].ID == customer_queue[0].ID):
                # pick table
                X = np.random.randint(12)
                # check full to find server picked
                for j in np.arange(12):
                    if(server_list[X].size == 0):
                        X=X+1
                        if(X>11):
                            X=0
                    else:
                        break
                # X is picked table
            
                #append chosen customer into server table
                server_list[X].cIDarray.append(customer_list[i].ID)
                customer_queue.pop(0)
                server_list[X].size=server_list[X].size - 1
                customer_list[i].sID = server_list[X].ID
                customer_list[i].entry_time = curr_time
                customer_list[i].departure_time = customer_list[i].entry_time + customer_list[i].dining_time
                customer_list[i].angryDay = 0
                customer_list[i].isWaiting = False
                total_server = total_server - 1
                print("Customer " + str(customer_list[i].ID) + " sit on table " + str(server_list[X].ID))
                print("Table " + str(server_list[X].ID) + " status: " + str(server_list[X].cIDarray))
                print("Remaining table is " + str(total_server))
                print("Departure time of customer " + str(customer_list[i].ID) + " is " + str(customer_list[i].departure_time))

        
    if(total_server == 0):
            print("The restaurant is full")    
            
    # return the time advanced at the end
    return curr_time + 1
    
def call_event(customer_list, customer_queue, server_list, curr_time, total_server, record):
    # extract and evaulate all jobs happens at current time
    # please take referece to the code in introduction to basic queueing system
    # step 0: you need to change the argument list above
    # step 1: arrival events, customers arrive and wait in the queue
    # step 2: departure events, customers finish dining and leave the servers (tables)
    # step 3: process the customer_queue, moving customers from queue to the servers (tables)
    return
    
def post_process(customer_list, customer_queue, server_list, record_result):
    # post-process the simulation at the end of ond day
    # step 0: you need to design the argument list above
    # step 1: process the customer queue, all customers in queue need to leave, they cannot dine in this canteen
    # step 1b: remember to handle the "hunger" issue, consecutive k days of "hunger"
    # step 2: process the servers (tables), depart the customers based on the increasing order of customers' leaving time
    # step 3: process the tests for flu, remove having p chance removing the customer from the system if they have flu
    # step 4: prcoess the record for the summary of one day simulation
    
    return

def showtime(curr_time):
    if(curr_time >= 0 and curr_time < 10):
        print("11:0" + str(curr_time))
    elif(curr_time >= 10 and curr_time < 60):
        print("11:" + str(curr_time))
    elif(curr_time >= 60 and curr_time < 70):
        print("12:0" + str(curr_time-60))
    elif(curr_time >= 70 and curr_time < 120):
        print("12:" + str(curr_time-60))
    elif(curr_time >= 120 and curr_time < 130):
        print("13:" + str(curr_time-120))
    elif(curr_time >= 130 and curr_time < 180):
        print("13:" + str(curr_time-120))
    elif(curr_time == 180):
        print("14:00")
    else:
        print(str(curr_time-180) + " minutes away from 14:00 ")

# main program

# initialization

# setting some default numbers
# finish all default numbers
seed_number = 0
D = 100
N = 100
T_size = 12
T_detail = [2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4]
p = 0.8
k = 2
termination_time = 180
total_server = 36
customer_list = []
server_list = []
for i in np.arange(T_size):
    tmp_server = Server(i,T_detail[i])
    server_list.append(tmp_server)

adding_record = "customer_seed0.csv"
adding_config = "config_seed0.csv"

Customer_file = "customer_v1.csv"
Result_file = "simulation_result.csv"
Summary_file = "simulation_summary.csv"

config = pd.DataFrame([])
config_data = [ ('D', D) , ('N', N) ,('T_size', T_size) ,('T_detail', T_detail) ,('p', p), ('k', k),('Customer_file' , Customer_file),('Result_file', Result_file),('Summary_file', Summary_file)]
config = pd.DataFrame(config_data)


record = pd.DataFrame([], columns = ["customer_id", "dining_time"])

# if the user specify the file name
# update the parameters based on the file
#if len(sys.argv) == 2:
    #config = pd.read_csv(sys.argv[1], header=None, index_col=0, skip_blank_lines=True)
    # print this out and see
    #print(int(config[1].D)) 
    #D = int(config[1].D)
    
    
    # remember to initialize the customer based on the Customer_file

    
#else:    
np.random.seed(seed_number)
    
    # initialize the customer based on random


# You need to initialize servers (tables)


# I believe these two lines are useful
record_result = pd.DataFrame([], columns = ["Day", "Time", "Event", "len(Queue)", "Number_dining", "Number_finish"])
record_summary = pd.DataFrame([], columns = ["Day", "ni", "Number_finish", "Hunger", "Hunger_removed"])


# main simulation
print("Start simulation")
seed_number = 1
while seed_number <= D:
    np.random.seed(seed_number)
    print("Starting of day " + str(seed_number))
    
    curr_time = 0
    customer_queue = []
    total_server = 36
    ni = 0
    nFinish = 0
    nHunger = 0
    HungerRemoved = []
    curr_day = seed_number
    
    # if we use random numbers for input
    if len(sys.argv) != 2:
        # generate the arrival time for the customers
        # first day to initialization 
        if(seed_number == 1):
            for i in np.arange(N):
                tmp_customer = Customer(i+1, int(np.random.uniform(120)), min(math.ceil(30+np.random.exponential(20)), 100))
                customer_list.append(tmp_customer)
            for i in np.arange(N):
                record=record.append({'customer_id' : i+1 , 'dining_time' : customer_list[i].dining_time} , ignore_index=True)
            arrival_time_result = [0] * len(record)
            for i in np.arange(N):
                arrival_time_result[i] = customer_list[i].arrival_time
            record.insert(loc=len(record.columns),column= "arrival_time_day " + str(1), value= arrival_time_result)
            arrival_time_result = [0] * len(record)
            ni = N
            
        #second days onward
        else:
            customer_list.sort(key=lambda X: X.ID)
            arrival_time_result = [0] * len(record)
            for i in np.arange(N):
                if(customer_list[i].isAngry == 1 or customer_list[i].isBlacklisted):
                    customer_list[i].arrival_time = -1
                else:
                    customer_list[i].arrival_time = int(np.random.uniform(120))
                    ni = ni + 1
                customer_list[i].entry_time = None
                customer_list[i].departure_time = -1
                customer_list[i].waiting_time = None
                customer_list[i].sID = None
                customer_list[i].isInside = False
                customer_list[i].isFinished = False
                customer_list[i].isWaiting = False
                arrival_time_result[i] = customer_list[i].arrival_time
            record.insert(loc=len(record.columns),column= "arrival_time_day " + str(seed_number), value= arrival_time_result)
            arrival_time_result = [0] * len(record)
            
        # sort the arrival time for the customers
        customer_list.sort(key=lambda X: X.arrival_time)
        test = 1
        
    #else:
        # set the arrival time for the customers, if needed
        #test = 1
    # think about in your design, do you have any initialization steps at the begining of each day
    # do the initialization for empty-and-idle state here
    
    # next day data initialization
    
    np.random.seed(seed_number+10000)

    while curr_time < termination_time or total_server < 36:
        # determine the time
        # you need to update the argument list
        
        showtime(curr_time)
        curr_time = timing_routine(customer_list, customer_queue, server_list, curr_time, termination_time, curr_day)
        
        # this is important, when it is 2:00 pm, customer in the queue cannot get into the canteen
        if curr_time >= termination_time:
            print("The restaurant is closed at day " + str(seed_number))
            
        if curr_time >= termination_time:
            x = 1
        # update different lists and log sheet
        # you need to update the argument list
        #customer_list, customer_queue, server_list, record_result = call_event(customer_list, customer_queue, server_list, curr_time, total_server, record_result)
    
    # clear the queue
    customer_list.sort(key=lambda X: X.ID)
    for i in np.arange(len(customer_queue)):
        customer_list[customer_queue[i].ID-1].angryDay = customer_list[customer_queue[i].ID-1].angryDay + 1
        print("This customer " + str(customer_list[customer_queue[i].ID-1].ID)  + " cannot enter restaurant for consecutive day " + str(customer_list[customer_queue[i].ID-1].angryDay))
        if(customer_list[customer_queue[i].ID-1].angryDay > 1):
            print("Customer " + str(customer_list[customer_queue[i].ID-1].ID) + " is too angry!")
            customer_list[customer_queue[i].ID-1].isAngry = 1
            nHunger = nHunger + 1
            if(customer_list[i].isRemoved == 0):
                HungerRemoved.append(customer_list[i].ID)
            customer_list[i].isRemoved = 1
    
    # post-processing and update the summary
    np.random.seed(seed_number+20000)
    
    summary_dict = pd.DataFrame({"Day":[seed_number],"ni":[ni], "Number_finish":[nFinish], "Hunger":[nHunger], "Hunger_removed":[HungerRemoved]})
    
    
    # You need to update the arguemnt list below
    #customer_list, customer_queue, server_list, record_result, summary_dict = post_process(customer_list, customer_queue, server_list, record_result)
    record_summary = record_summary.append(summary_dict, ignore_index=True)
    
    seed_number = seed_number + 1


if len(sys.argv) != 2:
    # output the customer record, if the customer record is generated by this program
    test = 1

# output result and summary
config.to_csv(adding_config, index = False)
record.to_csv(adding_record, index = False)

record_result.to_csv(Result_file, index = False)
record_summary.to_csv(Summary_file, index = False)

Start simulation
Starting of day 1
11:00
11:01
11:02
Customer 21 sit on table 1
Table 1 status: [21]
Remaining table is 35
Departure time of customer 21 is 60
11:03
11:04
Customer 80 sit on table 4
Table 4 status: [80]
Remaining table is 34
Departure time of customer 80 is 72
11:05
11:06
Customer 17 sit on table 6
Table 6 status: [17]
Remaining table is 33
Departure time of customer 17 is 52
11:07
Customer 35 sit on table 9
Table 9 status: [35]
Remaining table is 32
Departure time of customer 35 is 55
11:08
11:09
Customer 39 sit on table 3
Table 3 status: [39]
Remaining table is 31
Departure time of customer 39 is 48
Customer 57 sit on table 10
Table 10 status: [57]
Remaining table is 30
Departure time of customer 57 is 63
11:10
11:11
Customer 24 sit on table 3
Table 3 status: [39, 24]
Remaining table is 29
Departure time of customer 24 is 48
Customer 53 sit on table 8
Table 8 status: [53]
Remaining table is 28
Departure time of customer 53 is 61
Customer 98 sit on table 11
Table 11 st

The restaurant is full
12:52
Customer 59 is leaving at 112
Remaining table is 1
Customer 82 is leaving at 112
Remaining table is 2
Customer 37 sit on table 11
Table 11 status: [5, 96, 94, 37]
Remaining table is 1
Departure time of customer 37 is 175
Customer 65 sit on table 8
Table 8 status: [88, 9, 51, 65]
Remaining table is 0
Departure time of customer 65 is 149
The restaurant is full
12:53
The restaurant is full
12:54
Customer 47 is leaving at 114
Remaining table is 1
Customer 46 sit on table 6
Table 6 status: [64, 22, 27, 46]
Remaining table is 0
Departure time of customer 46 is 204
The restaurant is full
12:55
The restaurant is full
12:56
The restaurant is full
12:57
Customer 9 is leaving at 117
Remaining table is 1
Customer 23 sit on table 8
Table 8 status: [88, 51, 65, 23]
Remaining table is 0
Departure time of customer 23 is 159
The restaurant is full
12:58
Customer 25 is leaving at 118
Remaining table is 1
Customer 31 sit on table 3
Table 3 status: [83, 31]
Remaining table is 

Customer 100 sit on table 7
Table 7 status: [100]
Remaining table is 28
Departure time of customer 100 is 105
11:16
11:17
11:18
Customer 17 sit on table 9
Table 9 status: [50, 17]
Remaining table is 27
Departure time of customer 17 is 64
Customer 54 sit on table 2
Table 2 status: [54]
Remaining table is 26
Departure time of customer 54 is 102
11:19
Customer 19 sit on table 8
Table 8 status: [19]
Remaining table is 25
Departure time of customer 19 is 85
11:20
Customer 75 sit on table 2
Table 2 status: [54, 75]
Remaining table is 24
Departure time of customer 75 is 51
11:21
11:22
11:23
11:24
Customer 61 sit on table 11
Table 11 status: [61]
Remaining table is 23
Departure time of customer 61 is 55
11:25
Customer 37 sit on table 3
Table 3 status: [89, 37]
Remaining table is 22
Departure time of customer 37 is 88
Customer 79 sit on table 10
Table 10 status: [41, 79]
Remaining table is 21
Departure time of customer 79 is 73
11:26
Customer 16 sit on table 4
Table 4 status: [16]
Remaining tab

Customer 67 is leaving at 130
Remaining table is 2
Customer 13 sit on table 11
Table 11 status: [6, 7, 13]
Remaining table is 1
Departure time of customer 13 is 206
Customer 25 sit on table 11
Table 11 status: [6, 7, 13, 25]
Remaining table is 0
Departure time of customer 25 is 163
The restaurant is full
13:11
Customer 76 is leaving at 131
Remaining table is 1
Customer 5 is leaving at 131
Remaining table is 2
Customer 90 sit on table 7
Table 7 status: [91, 65, 95, 90]
Remaining table is 1
Departure time of customer 90 is 171
Customer 28 sit on table 8
Table 8 status: [87, 9, 27, 28]
Remaining table is 0
Departure time of customer 28 is 163
The restaurant is full
13:12
Customer 70 is leaving at 132
Remaining table is 1
Customer 47 is leaving at 132
Remaining table is 2
Customer 24 sit on table 6
Table 6 status: [84, 53, 15, 24]
Remaining table is 1
Departure time of customer 24 is 169
Customer 20 sit on table 10
Table 10 status: [10, 29, 64, 20]
Remaining table is 0
Departure time of cu

Customer 2 sit on table 3
Table 3 status: [2]
Remaining table is 17
Departure time of customer 2 is 73
Customer 57 sit on table 4
Table 4 status: [30, 57]
Remaining table is 16
Departure time of customer 57 is 89
Customer 87 sit on table 3
Table 3 status: [2, 87]
Remaining table is 15
Departure time of customer 87 is 75
11:36
11:37
Customer 22 sit on table 6
Table 6 status: [89, 34, 22]
Remaining table is 14
Departure time of customer 22 is 99
Customer 59 sit on table 6
Table 6 status: [89, 34, 22, 59]
Remaining table is 13
Departure time of customer 59 is 119
Customer 60 sit on table 7
Table 7 status: [5, 60]
Remaining table is 12
Departure time of customer 60 is 70
Customer 71 sit on table 7
Table 7 status: [5, 60, 71]
Remaining table is 11
Departure time of customer 71 is 70
11:38
Customer 85 sit on table 7
Table 7 status: [5, 60, 71, 85]
Remaining table is 10
Departure time of customer 85 is 85
11:39
Customer 15 sit on table 8
Table 8 status: [46, 73, 15]
Remaining table is 9
Depar

...

13:47
Customer 56 is leaving at 167
Remaining table is 18
13:48
Customer 58 is leaving at 168
Remaining table is 19
Customer 25 is leaving at 168
Remaining table is 20
13:49
13:50
13:51
Customer 62 is leaving at 171
Remaining table is 21
13:52
Customer 54 is leaving at 172
Remaining table is 22
13:53
13:54
Customer 86 is leaving at 174
Remaining table is 23
13:55
13:56
13:57
Customer 82 is leaving at 177
Remaining table is 24
13:58
Customer 37 is leaving at 178
Remaining table is 25
Customer 26 is leaving at 178
Remaining table is 26
13:59
Customer 66 is leaving at 179
Remaining table is 27
Customer 6 is leaving at 179
Remaining table is 28
The restaurant is closed at day 98
14:00
Customer 11 is leaving at 180
Remaining table is 29
The restaurant is closed at day 98
1 minutes away from 14:00 
Customer 45 is leaving at 181
Remaining table is 30
The restaurant is closed at day 98
2 minutes away from 14:00 
The restaurant is closed at day 98
3 minutes away from 14:00 
The restaurant is cl

Customer 1 sit on table 11
Table 11 status: [76, 91, 1]
Remaining table is 3
Departure time of customer 1 is 95
11:40
Customer 48 is leaving at 40
Remaining table is 4
11:41
11:42
Customer 100 sit on table 2
Table 2 status: [37, 100]
Remaining table is 3
Departure time of customer 100 is 132
11:43
Customer 27 sit on table 11
Table 11 status: [76, 91, 1, 27]
Remaining table is 2
Departure time of customer 27 is 80
Customer 44 sit on table 9
Table 9 status: [9, 3, 64, 44]
Remaining table is 1
Departure time of customer 44 is 140
Customer 62 sit on table 5
Table 5 status: [78, 62]
Remaining table is 0
Departure time of customer 62 is 79
The restaurant is full
11:44
The restaurant is full
11:45
The restaurant is full
11:46
Customer 75 is leaving at 46
Remaining table is 1
Customer 65 sit on table 4
Table 4 status: [59, 65]
Remaining table is 0
Departure time of customer 65 is 83
The restaurant is full
11:47
The restaurant is full
11:48
The restaurant is full
11:49
Customer 9 is leaving at 

Customer 73 is leaving at 122
Remaining table is 3
Customer 19 sit on table 4
Table 4 status: [35, 19]
Remaining table is 2
Departure time of customer 19 is 188
Customer 28 sit on table 6
Table 6 status: [67, 86, 93, 28]
Remaining table is 1
Departure time of customer 28 is 154
Customer 30 sit on table 10
Table 10 status: [98, 96, 31, 30]
Remaining table is 0
Departure time of customer 30 is 177
The restaurant is full
13:3
The restaurant is full
13:4
The restaurant is full
13:5
The restaurant is full
13:6
The restaurant is full
13:7
The restaurant is full
13:8
Customer 11 is leaving at 128
Remaining table is 1
Customer 67 is leaving at 128
Remaining table is 2
Customer 34 sit on table 6
Table 6 status: [86, 93, 28, 34]
Remaining table is 1
Departure time of customer 34 is 173
Customer 60 sit on table 7
Table 7 status: [50, 53, 95, 60]
Remaining table is 0
Departure time of customer 60 is 161
The restaurant is full
13:9
The restaurant is full
13:10
Customer 57 is leaving at 130
Remainin

Customer 52 sit on table 8
Table 8 status: [74, 52]
Remaining table is 23
Departure time of customer 52 is 53
11:15
11:16
11:17
11:18
Customer 50 sit on table 2
Table 2 status: [11, 50]
Remaining table is 22
Departure time of customer 50 is 68
11:19
Customer 4 sit on table 8
Table 8 status: [74, 52, 4]
Remaining table is 21
Departure time of customer 4 is 58
11:20
11:21
Customer 8 sit on table 6
Table 6 status: [16, 8]
Remaining table is 20
Departure time of customer 8 is 74
11:22
Customer 19 sit on table 11
Table 11 status: [84, 19]
Remaining table is 19
Departure time of customer 19 is 88
Customer 23 sit on table 10
Table 10 status: [51, 23]
Remaining table is 18
Departure time of customer 23 is 64
11:23
Customer 17 sit on table 5
Table 5 status: [39, 17]
Remaining table is 17
Departure time of customer 17 is 69
11:24
Customer 87 sit on table 3
Table 3 status: [87]
Remaining table is 16
Departure time of customer 87 is 64
11:25
Customer 29 sit on table 9
Table 9 status: [65, 29]
Rema

The restaurant is full
12:47
Customer 66 is leaving at 107
Remaining table is 1
Customer 40 is leaving at 107
Remaining table is 2
Customer 58 sit on table 11
Table 11 status: [43, 1, 89, 58]
Remaining table is 1
Departure time of customer 58 is 166
Customer 15 sit on table 4
Table 4 status: [49, 15]
Remaining table is 0
Departure time of customer 15 is 180
The restaurant is full
12:48
The restaurant is full
12:49
Customer 68 is leaving at 109
Remaining table is 1
Customer 12 sit on table 3
Table 3 status: [95, 12]
Remaining table is 0
Departure time of customer 12 is 163
The restaurant is full
12:50
The restaurant is full
12:51
Customer 1 is leaving at 111
Remaining table is 1
Customer 45 sit on table 11
Table 11 status: [43, 89, 58, 45]
Remaining table is 0
Departure time of customer 45 is 161
The restaurant is full
12:52
The restaurant is full
12:53
Customer 11 is leaving at 113
Remaining table is 1
Customer 7 is leaving at 113
Remaining table is 2
Customer 70 sit on table 2
Table 2